<div>
     <div>
        <img src="./data/isel_logo.png" width="200" height="200" align="left">
    </div>
    <div>
        <h1>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h1>
        <p>Trabalho prático 2</p>
        <p>Authors:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>9 – Maio – 2021</p>
    </div>
</div>

<a id="index"></a>
# Index
- [Tabela de Huffman](#tabela_huffman)
    - [bytes_frequency](#bytes_frequency)
    - [node](#node)
    - [create_huff_tree](#create_huff_tree)
    - [node2huff_dictionary](#node2huff_dictionary)
    - [gen_huff_table](#gen_huff_table)
- [Codificador de Huffman](#codificador_huffman)
    - [encode_huff](#encode_huff)
- [Descodificador de Huffman](#descodificador_huffman)
    - [decode_huff](#decode_huff)


# Importar bibliotecas

In [2]:
import os
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd() # current work diretory

# Importar dados para teste

In [3]:
dec_universal_IDH = np.fromfile(f"{cwd}/data/DecUniversalDH.txt",dtype=np.int8)
lena_color = np.fromfile(f"{cwd}/data/LenaColor.tif",dtype=np.int8)
lena_gray = cv2.imread(f"{cwd}/data/LenaGray.tif");

 <a id="tabela_huffman"></a>

# Tabela de Huffman

 <a id="node"></a>


## Função que cria um dicionário com par chave-valor : símbolo-frequência

Função lê um ficheiro, byte a byte, e retorna um dicionário onde cada símbolo terá como respondência a sua frequência no ficheiro. 

In [57]:
# return dicionary {byte : frequency}
def bytes_frequency(file):
    d = dict()
    for byte in file:
        d[byte] = d.setdefault(byte, 0) + 1
    return d

 <a id="bytes_frequency"></a>

## Classe que representa um nó de Huffman

Cada nó contêm a seguinte informação:
* o símbolo
* a frequência do símbolo
* uma ligação para a esquerda e para a direita para os seus nós filhos
* o valor de huffman atribuído quando o nó toma uma direção

In [62]:
# Huffman Node
class huff_node:
    def __init__(self, symbol, freq, left=None, right=None):
        # symbol
        self.symbol = symbol
        # frequency of symbol
        self.freq = freq
        # node left of current node
        self.left = left
        # node right of current node
        self.right = right
        # tree direction (0/1)
        self.huff = ''

 <a id="create_huff_tree"></a>

## Função que cria uma árvore de nós de Huffman a partir de um dicionário

1. Com o valor de cada chave única presente no dicionário, são criados nós e colocados numa lista;
2. São retirados os dois símbolos com menor frequência da lista, atribuindo-lhes o valor de 0 ou 1, e mergem-se esses dois símbolos num só, somando as suas freqûencias; 
3. O novo nó é adicionado a lista;
4. O prodecimento repete-se até enquanto o número de nós for superior a 1;
5. A função retorna o nó raiz.

In [61]:
# Create HuffmanTree from dictionary of {symbol : frequency}
def create_huff_tree(dictionary):
    list_of_nodes = []
    for symb, freq in dictionary.items():
        list_of_nodes.append(huff_node(symb, freq))
    
    while len(list_of_nodes) > 1:
        list_of_nodes = sorted(list_of_nodes, key=lambda n: n.freq)
        # pop the 2 smallest nodes
        left = list_of_nodes.pop(0)
        right = list_of_nodes.pop(0)
        # assign directional value
        left.huff = 0
        right.huff = 1
        # combine the 2 smallest nodes to create new node as their parent
        newNode = huff_node(left.symbol+right.symbol, left.freq+right.freq, left, right)
        list_of_nodes.append(newNode)
        
    return list_of_nodes[0]

<a id="node2huff_dictionary"></a>

## Função que a partiz de um nó raiz de Huffman gera uma tabela de Huffman

Recursivamente percorre toda a arvóre de nós de Huffman, concatenando os valores de Huffman dos nós enquando não chega aos nós folha.

In [7]:
def node2huff_dictionary(node, dictionary, value = ""):
    new_value = value + str(node.huff)
    if(node.left):
        node2huff_dicitionary(node.left, dictionary, new_value)
    if(node.right):
        node2huff_dicitionary(node.right, dictionary, new_value)
    # if node is leaf
    if(not node.left and not node.right):
        dictionary[node.symbol] = new_value

<a id="gen_huff_table"></a>

## Função que agrega todas as chamadas para gerar a tabela de Huffman

In [18]:
def gen_huff_table(file):
    d = bytes_frequency(file)
    root = create_huff_tree(d)
    huff_table = dict()
    node2huff_dictionary(root, huff_table)
    return huff_table

In [60]:
t_start = time()
huff_table = gen_huff_table("otorrinolaringologista")
t_end = time()
print("time: ", t_end - t_start)
print(huff_table)

time:  0.0
{'a': '000', 'g': '001', 'o': '01', 'r': '100', 'i': '101', 's': '1100', 't': '1101', 'n': '1110', 'l': '1111'}


 <a id="codificador_huffman"></a>

# Codificador de Huffman

Dada uma mensagem e uma tabela de Huffman, codifica uma mensagem.

<a id="encode_huff"></a>

In [53]:
def encode_huff(file, huff_table):
    output = ""
    for byte in file:
        output += huff_table[byte]
    return output # int(output, 2)

In [54]:
decoded_message = encode_huff("otorrinolaringologista", huff_table)

print(decoded_message)

01110101100100101111001111100010010111100010111110100110111001101000


 <a id="descodificador_huffman"></a>

# Descodificador de Huffman

A partir deada uma sequência de bits (mensagem codificada) e uma tabela de huffman, retorne uma mensagem descodificada.

<a id="decode_huff"></a>

In [48]:
def decode_huff (encoded_message, huff_table):
    inverted_huff_table = dict(map(reversed, huff_table.items()))
    buffer = ""
    output = ""
    for byte in encoded_message:
        buffer += byte
        if buffer in inverted_huff_table:
            output += inverted_huff_table[buffer]
            buffer = ""
    return output

In [55]:
decoded_msg = decode_huff("01110101100100101111001111100010010111100010111110100110111001101000", huff_table)
print(decoded_msg)

otorrinolaringologista


In [ ]:
def write2file(encoded_msg, filename):

In [ ]:
def read2array(filename):
    with open(filename, "rb") as f:
        byte = f.read(1)
        while byte != b"":
            # do stuff
            byte = f.read(1)